In [2]:
import random
hello_dict = { 0 : "hello!", 1 : "I'm here", 2 : "hey!" ,3 : "what's up?" }
Goodbye_dict = { 0 : "Goodbye!", 1 : "Nice to see you", 2 : "ok!" ,3 : "Well.." }
num = random.randrange(4)
hello_dict[num]

'hey!'